In [1]:
import numpy as np
import Simplagion_functions_SIR as SM
from tqdm.notebook import tqdm
#import seaborn as sns
import matplotlib.pyplot as plt
import csv
import networkx as nx
import random
import pickle

%load_ext autoreload
%autoreload 2

In [2]:
weighted = True

In [3]:
#dataset = 'hospital'
#dataset = 'high_school'
#dataset = 'workplace'
#dataset = 'conference'
dataset = 'pr_school'



edgelist_name = '../../data/pairwise/' + dataset + '_strength_edgelist_pairwise_sorted_norm.pkl'
with open(edgelist_name, 'rb') as f:
    edgelist = pickle.load(f)

edgelist3_name = '../../data/all_triangles/' + dataset + '_strength_edgelist3_sorted_norm.pkl'
with open(edgelist3_name, 'rb') as f:
    edgelist3 = pickle.load(f)

edgelist2_name = '../../data/all_HO/' + dataset + '_strength_edgelist2_sorted_norm.pkl'
with open(edgelist2_name, 'rb') as f:
    edgelist2 = pickle.load(f)

nodes_list = np.unique(list(edgelist.keys()))

print(nodes_list)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241]


In [4]:
def save_on_csv(filename, variable_list, writing_operation):
    with open(filename, writing_operation) as csvfile:
        writer = csv.writer(csvfile)
        try:
            [writer.writerow(s) for s in variable_list]
        except:
            writer.writerow(variable_list)
    csvfile.close()

In [5]:
G = nx.Graph()
for edge in list(edgelist.keys()):
    G.add_weighted_edges_from([tuple((edge[0],edge[1],edgelist[edge]))])
nb_nodes = len(list(G.nodes))

### Simulate:

Salvo solo la media sui run dei pesi:

In [6]:
#filename = 'results/store_results.txt'
#file = open(filename, 'a')
#file.write('beta | betaT | mu  |  nS  |  nI  |  nR  | end_t \n')
#file.close()

In [7]:
mu = 0.1
if dataset == 'pr_school':
    beta_betaT_range = [(0.07,7),
                        #(0.1,7),
                        (0.2,7),
                        (0.1,5),
                        (0.15,5),
                        (0.2,5),
                        #(0.2,3),
                        (0.2,1),
                        #(0.08,7),
                        #(0.11,6),
                        ]

In [8]:
#seed_size = int(round(nb_nodes/10))
seed_size = 1

In [9]:
final_result = 'inf_tree'
save_temp_ev = False

# If final_result = 'inf_tree'

In [10]:
times = 1000

In [11]:
pat0 = -1 # tanto non si usa

In [12]:
#random.seed(1)
print(dataset)
sm = SM.Simplagion_model(edgelist,edgelist3,nodes_list,weighted,final_result)
for (beta,betaT) in beta_betaT_range:
    print('beta = ',beta,'betaT = ',betaT,'mu = ',mu,'seed_size =',seed_size)
    C_L_list = np.full(len(edgelist)*2,0)
    C_T_list = np.full(len(edgelist)*2,0)
    t_i = [[] for i in range(nb_nodes)]
    for n in tqdm(range(times)):
        sm.initialize_time0(seed_size)
        C_L_list_tmp, C_T_list_tmp, t_i, a = sm.simulate(beta,betaT,mu, save_temp_ev,t_i)
        #print('Soglia superata: ', flag_nb_inf)
        C_L_list = C_L_list + C_L_list_tmp
        C_T_list = C_T_list + C_T_list_tmp

    # save infection pattern:
    C_L_list = C_L_list/times
    C_T_list = C_T_list/times
    if mu > 0:
        if weighted:
            file_nameL = "results/inf_treeSIR/CL_mean_weighted_%s_beta_%.3f_betaT_%.3f_mu_%.2f_init_%d.csv"%(dataset,beta,betaT,mu,seed_size)
            file_nameT = "results/inf_treeSIR/CT_mean_weighted_%s_beta_%.3f_betaT_%.3f_mu_%.2f_init_%d.csv"%(dataset,beta,betaT,mu,seed_size)
        else:
            file_nameL = "results/inf_treeSIR/CL_mean_%s_beta_%.3f_betaT_%.3f_mu_%.2f_init_%d.csv"%(dataset,beta,betaT,mu)
            file_nameT = "results/inf_treeSIR/CT_mean_%s_beta_%.3f_betaT_%.3f_mu_%.2f_init_%d.csv"%(dataset,beta,betaT,mu)
    save_on_csv(file_nameL,C_L_list,'a+')
    save_on_csv(file_nameT,C_T_list,'a+')

    # save t_i mean:
    '''
    t_i_mean = np.empty(nb_nodes)
    for node in range(len(t_i)):
        t_i_mean[node] = np.mean(t_i[node])
    if mu > 0: 
        if weighted:
            file_name = "results/inf_time/t_mean_weighted_%s_beta_%.3f_betaT_%.3f_mu_%.3f_init_%d.csv"%(dataset,beta,betaT,mu,seed_size)
    save_on_csv(file_name,t_i_mean,'w')
    '''

pr_school
beta =  0.07 betaT =  7 mu =  0.1 seed_size = 1



beta =  0.2 betaT =  7 mu =  0.1 seed_size = 1



beta =  0.1 betaT =  5 mu =  0.1 seed_size = 1



beta =  0.15 betaT =  5 mu =  0.1 seed_size = 1



beta =  0.2 betaT =  5 mu =  0.1 seed_size = 1



beta =  0.2 betaT =  1 mu =  0.1 seed_size = 1


In [13]:
t_i = [[] for i in range(nb_nodes)]

In [14]:
#######################

In [15]:
def load_C_mean(filename):
    with open(filename, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            C = np.array([float(i) for i in row])
    return C

In [16]:

for betaT in betaT_range:
    for beta in beta_range:
        beta = np.round(beta,4)
        betaT = np.round(betaT,4)
        print('beta=',beta,'betaT',betaT,'seed_size',seed_size)
        filenameL = "results/inf_treeSIR/CL_mean_weighted_%s_beta_%.3f_betaT_%.3f_mu_%.2f_init_%d.csv"%(dataset,beta,betaT,mu,seed_size)
        filenameT = "results/inf_treeSIR/CT_mean_weighted_%s_beta_%.3f_betaT_%.3f_mu_%.2f_init_%d.csv"%(dataset,beta,betaT,mu,seed_size)
        CL = load_C_mean(filenameL)
        CT = load_C_mean(filenameT)

NameError: name 'betaT_range' is not defined

In [ ]:
np.sum(CL), np.sum(CT)